# Tool/Function Chaining

In [1]:
cd ..

c:\Users\suriya\Documents\projects\Solving-Hard-Problems-With-LLMs


c:\Users\suriya\Documents\projects\Solving-Hard-Problems-With-LLMs\en-llm\Lib\site-packages\IPython\core\magics\osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [2]:
%load_ext autoreload
%autoreload 2

## Setup openai API

In [4]:
import openai
# openai_api_key = open(".secrets/openai-api_key.txt").read()
# or set an env variable

## Setup tools

I've made a few changes.

`computeStatistic` now takes a pandas dataframe as input.

So, the flow looks like: `getHistoricalPrice(start, end)` -> `dataframe` -> `computeStatistic(dataframe, statistic)`

In [20]:
tools = [{
        'type': 'function',
        'function': {
            'name': 'getHistoricalPrice',
            'description': '''
            Returns historical stock price data (dataframe) given start and end dates.
            Example: Give me last month\'s stock price data.
            '''
            ,
            'parameters': {
            'type': 'object',
            'properties': {
                'start': {
                'type': 'string',
                'description': 'Start Date in YYYY-MM-DD format',
                },
                'end': {
                'type': 'string',
                'description': 'End Date in YYYY-MM-DD format',
                },
            },
            'required': ['start', 'end'],
            },
        },
        },
        {
        'type': 'function',
        'function': {
            'name': 'computeStatistic',
            'description': '''
            Given stock price data (dataframe) compute a summary statistic represented by statistic argument.
            Example: "What is the maximum stock price this year?".
            '''
            ,
            'parameters': {
            'type': 'object',
            'properties': {
                'stockPrice': {
                'type': 'object',
                'description': 'Stock price data given by pandas.DataFrame object',
                },
                'statistic': {
                'type': 'string',
                'description': 'Summary statistic to compute. Options include "min", "max", and, "mean".',
                },
            },
            'required': ['stockPrice', 'statistic'],
            },
        },
        },
    ]


## Define functions

In [21]:
from datetime import datetime
DATE_FORMAT = "%Y-%m-%d"


def today():
    return datetime.now().strftime(DATE_FORMAT)

In [56]:
import yfinance as yf
import pandas as pd
TICKER = "AAPL"

def getHistoricalPrice(start: str, end: str):
    df = yf.download(TICKER, start, end)
    return df["Close"]


def computeStatistic(stockPrice: pd.DataFrame, statistic: str="mean"):
    return {
        "mean": stockPrice.mean(),
        "max": stockPrice.max(),
        "min": stockPrice.min()
    }[statistic].item()

start, end = "2024-08-01", "2024-08-31"
df = getHistoricalPrice(start, end)
computeStatistic(df, statistic="min")

[*********************100%***********************]  1 of 1 completed


207.22999572753906

## Update Function Calling

When `tool_calls` is a list of more than one function, chain them.

In [68]:
import json

def call_fns(tool_calls):
    for fn in tool_calls:
        print(fn.function)
    fndef = {
        "getHistoricalPrice": getHistoricalPrice,
        "computeStatistic": computeStatistic
    }

    x = None
    for fn in tool_calls:
        args = json.loads(fn.function.arguments)
        name = fn.function.name
        if x is None:
            print("name:", name, "args:", args)
            x = fndef[name](**args)
        else:
            x = fndef[name](x, **args)
    
    return x

## Generate Response

In [69]:
def generate_response(query):
    response = openai.chat.completions.create(
        model="gpt-4o",
        messages=[{"role": "system", "content": f"today's date is {today()}"}, {'role': 'user', 'content': query}],
        tools=tools,
    )
    message = response.choices[0].message
    if message.content:
        return message.content
    
    return call_fns(message.tool_calls)

In [72]:
response = generate_response("What's the maximum stock price last year?")

[*********************100%***********************]  1 of 1 completed

Function(arguments='{"start": "2023-01-01", "end": "2023-12-31"}', name='getHistoricalPrice')
Function(arguments='{"statistic": "max"}', name='computeStatistic')
name: getHistoricalPrice args: {'start': '2023-01-01', 'end': '2023-12-31'}


In [73]:
response

198.11000061035156

## Conclusion

Chaining multiple function calls was easier than I expected.

My observations from running this example multiple times, tells me it fails about half the time.

How do we make it reliable? 